In [6]:
from itertools import chain

import numpy as np
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import dgl
import dgl.data
import dgl.nn as gnn
import dgl.function as fn

Workflow:
1. Имеющиеся связи - положительные примеры
2. Нужно насэплить несуществующих связей - это будут негативные примеры. Задача предсказания связей состоит в сравнении оценок вероятности существования связи между парами узлов, которые действительно связаны, и между парами узлов, выбранных случайно. 
3. Разделить полученные примеры на обучающую и тестовую выборку
4. Решить задачу бинарной классификации

In [16]:
dataset = dgl.data.CoraGraphDataset()
G = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [36]:
# создаем обучающую и тестовую выборку
u, v = G.edges()

# positive triples
eids = np.random.permutation(np.arange(G.num_edges()))
test_size = int(len(eids) * .1)
train_size = len(eids) - test_size

test_pos_u, test_pos_v = u[:test_size], v[:test_size]
train_pos_u, train_pos_v = u[test_size:], v[test_size:]

# negative triples
# adj[i, j] == 1 iif i->v
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
# adj_neg[i, j] == 1 iif adj[i, j] = 0 and i != j
adj_neg = 1 - adj.todense() - np.eye(G.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), G.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[train_size:]], neg_v[neg_eids[train_size:]]

# удаляем ребра из тестового множества
train_G = dgl.remove_edges(G, eids[:test_size])

In [13]:
class GCN(nn.Module):
    '''Для каждого узла возвращает вектор (эмбеддинг) длины n_hidden. На основе близости двух эмбеддингов узлов
    принимается решение о том, существует связь или нет'''
    def __init__(self, n_inputs, n_hidden):
        super().__init__()
        self.conv1 = gnn.SAGEConv(n_inputs, n_hidden, aggregator_type='mean')
        self.conv2 = gnn.SAGEConv(n_hidden, n_hidden, aggregator_type='mean')

    def forward(self, G, features):
        out = F.relu(self.conv1(G, features))
        out = self.conv2(G, out)
        return out

In [14]:
class DotPredictor(nn.Module):
    def forward(self, G, features):
        with G.local_scope():
            G.ndata['h'] = features
            G.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return G.edata['score'].squeeze()

Для работы создадим 4 графа: 2 из них на основе обучающего и тестового множества, содержащих положительные примеры; и 2 - на основе множеств, содержащих негативные примеры. Для удобства явно указываем, что в каждом из них одинаковое кол-во вершин.

Имея на руках графы, можем воспользоваться методом `apply_edges` и быстренько посчитать фичи для ребер на основе эмбеддингов узлов.

In [52]:
train_pos_G = dgl.graph((train_pos_u, train_pos_v), num_nodes=G.num_nodes())
train_neg_G = dgl.graph((train_neg_u, train_neg_v), num_nodes=G.num_nodes())

test_pos_G = dgl.graph((test_pos_u, test_pos_v), num_nodes=G.num_nodes())
test_neg_G = dgl.graph((test_neg_u, test_neg_v), num_nodes=G.num_nodes())


In [56]:
n_inputs = G.ndata['feat'].shape[1]
n_hidden = 16
n_epochs = 100
model = GCN(n_inputs, n_hidden)
predictor = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones_like(pos_score), torch.zeros_like(neg_score)])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones_like(pos_score), torch.zeros_like(neg_score)])
    return roc_auc_score(labels, scores)

optimizer = optim.Adam(chain(model.parameters(), predictor.parameters()), lr=.01)

for epoch in range(n_epochs):
    # forward
    embeddings = model(train_G, train_G.ndata['feat'])
    pos_score = predictor(train_pos_G, embeddings)
    neg_score = predictor(train_neg_G, embeddings)
    loss = compute_loss(pos_score, neg_score)
    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if not epoch % 5:
        print('In epoch {}, loss: {}'.format(epoch, loss))
with torch.no_grad():
    pos_score = predictor(test_pos_G, embeddings)
    neg_score = predictor(test_neg_G, embeddings)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.6911070942878723
In epoch 5, loss: 0.36575984954833984
In epoch 10, loss: 0.3100879490375519
In epoch 15, loss: 0.3257092535495758
In epoch 20, loss: 0.30938130617141724
In epoch 25, loss: 0.28581932187080383
In epoch 30, loss: 0.28284579515457153
In epoch 35, loss: 0.2713790833950043
In epoch 40, loss: 0.2587047219276428
In epoch 45, loss: 0.24599483609199524
In epoch 50, loss: 0.2322506606578827
In epoch 55, loss: 0.21729446947574615
In epoch 60, loss: 0.20117148756980896
In epoch 65, loss: 0.18405020236968994
In epoch 70, loss: 0.16695138812065125
In epoch 75, loss: 0.1509188413619995
In epoch 80, loss: 0.13618172705173492
In epoch 85, loss: 0.12221866846084595
In epoch 90, loss: 0.10906071215867996
In epoch 95, loss: 0.09686066210269928
AUC 0.707791828575279


Еще один вариант negative sampling
Отличия:
1. Нет разделения на train/test
2. Для каждого позитивного примера строится `k` негативных

In [18]:
def construct_negative_graph(G, k):
    u, v = G.edges()
    neg_u = u.repeat_interleave(k)
    neg_v = torch.randint(0, G.num_nodes(), (len(neg_u),))
    return dgl.graph((neg_u, neg_v), num_nodes=G.num_nodes())

In [29]:
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden):
        super().__init__()
        self.conv1 = gnn.SAGEConv(n_inputs, n_hidden, aggregator_type='mean', activation=F.relu)
        self.conv2 = gnn.SAGEConv(n_hidden, n_hidden, aggregator_type='mean')
        self.predictor = DotPredictor()
    
    def forward(self, G, nG, G_features):
        out = self.conv1(G, G_features)
        out = self.conv2(G, out)
        pred_pos = self.predictor(G, out)
        pred_neg = self.predictor(nG, out)
        return pred_pos, pred_neg

In [32]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones_like(pos_score), torch.zeros_like(neg_score)])
    return F.binary_cross_entropy_with_logits(scores, labels)

In [37]:
n_inputs = G.ndata['feat'].shape[1]
n_hidden = 16
n_epochs = 1000

model = GCN(n_inputs, n_hidden)
optimizer = optim.Adam(model.parameters(), lr=.001)

for epoch in range(n_epochs):
    # forward
    nG = construct_negative_graph(G, k=3)
    pos_score, neg_score = model(G, nG, G.ndata['feat'])
    loss = compute_loss(pos_score, neg_score)
    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if not epoch % 50:
        print('In epoch {}, loss: {}'.format(epoch, loss))

In epoch 0, loss: 0.6945202350616455
In epoch 50, loss: 0.6912407279014587
In epoch 100, loss: 0.6608276963233948
In epoch 150, loss: 0.638744592666626
In epoch 200, loss: 0.6267092823982239
In epoch 250, loss: 0.6215561032295227
In epoch 300, loss: 0.6158761978149414
In epoch 350, loss: 0.6108839511871338
In epoch 400, loss: 0.6088335514068604
In epoch 450, loss: 0.6077801585197449
In epoch 500, loss: 0.6053219437599182
In epoch 550, loss: 0.604031503200531
In epoch 600, loss: 0.6006120443344116
In epoch 650, loss: 0.598950207233429
In epoch 700, loss: 0.595956027507782
In epoch 750, loss: 0.5948807597160339
In epoch 800, loss: 0.5922162532806396
In epoch 850, loss: 0.5932337045669556
In epoch 900, loss: 0.5919910669326782
In epoch 950, loss: 0.5914747714996338
